## Example #1 (google search):
How to use EvalTask.evaluate() with prompt_template?

In [ ]:
import pandas as pd
from vertexai.preview.evaluation import EvalTask

# Prepare your evaluation dataset
eval_dataset = pd.DataFrame({
    "context": ["Article about topic 1", "Article about topic 2"],
    "instruction": ["Summarize this article", "Explain the key points"],
    "reference": ["Reference summary 1", "Reference explanation 2"],
})

# Define your prompt template
prompt_template = "{instruction}. Article: {context}. Summary:"

# Define your model (replace with your actual model)
# For example, a custom prediction function:
def custom_model_fn(input: str) -> str:
  # Your model inference logic here
  return "model response"

# Or a Vertex AI model:
# model = your_vertex_ai_model

# Define your evaluation metrics
metrics = ["summarization_quality"]

# Create and run the EvalTask
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=metrics,
)

eval_result = eval_task.evaluate(
    model=custom_model_fn,  # Or your Vertex AI model
    prompt_template=prompt_template,
)

# View and interpret the results
print(eval_result)

## Example #2 (Gemini)

context:

Evaluate Gen AI Model and Agent Performance in GCP



question:

how to use EvalTask.evaluate()  by using Pointwise summarization quality (identifying by MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY) with prompt_template?

In [ ]:
import vertexai
from vertexai.evaluation import EvalTask, MetricPromptTemplateExamples
from vertexai.generative_models import GenerativeModel
import pandas as pd

PROJECT_ID = "???"
LOCATION = "???" # e.g., "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)

# Prepare your evaluation dataset
eval_dataset = pd.DataFrame({
    "context": [
        "The quick brown fox jumps over the lazy dog. This sentence is often used for testing typing speed and displaying all letters of the alphabet.",
        "The Amazon rainforest is the largest rainforest in the world. It is home to an incredible diversity of plants and animals, and plays a crucial role in regulating the Earth's climate."
    ],
    "instruction": [
        "Summarize the following text concisely.",
        "Write a brief summary of the provided article."
    ],
    "reference": [
        "The quick brown fox jumps over the lazy dog is a pangram used for typing tests.",
        "The Amazon is the world's largest rainforest, vital for global climate and biodiversity."
    ]
})


# Define your prompt template
prompt_template = "Instruction: {instruction}\nArticle: {context}\nSummary:"


# Define your model (replace with your actual model)
# Assuming you have a GenerativeModel instance
# For example, using Gemini 1.5 Pro:
model = GenerativeModel("gemini-1.5-pro")

eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY],
)

result = eval_task.evaluate(
    model=model,
    prompt_template=prompt_template,
    # Optional: You can specify an experiment_run_name for better tracking in Vertex AI Experiments
    # experiment_run_name="my-summarization-eval-run"
)

# Access the results
print(result.metrics)
print(result.to_dataframe())



## Example #2 (adjusted w/ GENAI098 requirements)

In [ ]:
import vertexai
from vertexai.evaluation import EvalTask, MetricPromptTemplateExamples
from vertexai.generative_models import GenerativeModel
import pandas as pd

# --- Configuration ---
PROJECT_ID = "your-gcp-project-id"  # <--- REPLACE WITH YOUR GCP PROJECT ID
LOCATION = "us-central1"           # <--- REPLACE WITH YOUR GCP REGION (e.g., "us-central1")

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

# --- Define Prompt Template ---
prompt_template = "# System_prompt\n{system_prompt} # Question\n{question} # Description {description}"

# --- Your Data for Evaluation ---
# These lists will become columns in your evaluation dataset.
# Make sure they all have the same number of elements for the DataFrame.

system_prompt_data = [
    "You are an retail domestic merchandise expert",
    "You are an retail domestic merchandise expert",
    "You are an retail domestic merchandise expert",
    "You are an retail domestic merchandise expert",
    "You are an retail domestic merchandise expert"
]

question_data = [
    "Provide a one sentence summary of the following text",
    "Provide a one sentence summary of the following text",
    "Provide a one sentence summary of the following text",
    "Provide a one sentence summary of the following text",
    "Provide a one sentence summary of the following text"
]

description_data = [
    "Men’s Blue Dress Shorts Elevate your warm-weather wardrobe with these tailored men's blue dress shorts — where polished style meets everyday comfort. Designed with a modern slim fit and cut from a lightweight, breathable cotton-blend fabric, these shorts offer a refined silhouette perfect for smart-casual settings. Featuring a flat front, belt loops, and discreet side and back pockets, they pair effortlessly with a crisp button-down or a relaxed polo. Whether you're headed to a summer wedding, rooftop brunch, or casual Friday at the office, these shorts strike the perfect balance between laid-back and sophisticated",
    "Summer Floral Dress. Breathe life into your summer wardrobe with this effortlessly elegant floral midi dress. Crafted from lightweight, breathable fabric, this dress is designed to keep you cool and confident from sunny brunches to sunset strolls. The vibrant floral print pops against soft pastels, while the flattering silhouette — complete with a cinched waist and flowing A-line skirt — moves beautifully with every step. Featuring delicate spaghetti straps, a sweetheart neckline, and subtle ruffle detailing, this dress is the perfect balance of feminine charm and relaxed ease. Style it with strappy sandals for a daytime look or elevate it with wedges and a clutch for those golden-hour moments.",
    "Outdoor Garden Furniture Transform your backyard into a personal oasis with this elegant garden furniture set designed for comfort, durability, and timeless style. Whether you're hosting a summer soirée, enjoying a quiet morning coffee, or stretching out under the stars, this set offers the perfect blend of relaxation and sophistication. Crafted with weather-resistant materials and plush, all-season cushions, it’s built to withstand the elements while keeping you cozy. The modern, neutral design complements any garden, patio, or balcony — making it a seamless fit for both small spaces and open-air retreats.",
    "OLED 4K Ultra HD Smart TV. Step into the future of home entertainment with breathtaking clarity, vibrant color, and cinematic sound. This OLED 4K Ultra HD Smart TV transforms your living room into a private theater — where every scene comes alive with lifelike detail and stunning contrast. With self-lit pixels that turn on and off individually, OLED delivers perfect blacks and infinite contrast, revealing depth and dimension that LED TVs simply can’t match. Whether you're streaming your favorite show, gaming at high frame rates, or watching the big game in dazzling clarity, every moment is immersive. Powered by a next-gen AI processor, it automatically adjusts picture and sound based on your environment and content, so you always get the optimal viewing experience. And with an ultra-slim bezel and sleek modern design, it’s not just a TV — it’s a centerpiece.",
    "Smartwash Dishwasher. Let your kitchen work for you. Say goodbye to scrubbing and soaking — the SmartWash Dishwasher delivers a powerful, whisper-quiet clean that saves you time, energy, and water. Whether it’s post-dinner chaos or a pile of party plates, this dishwasher handles it all with precision and polish. With advanced spray technology, high-temperature sanitization, and intuitive smart cycles, every dish comes out sparkling — from delicate wine glasses to stubborn, baked-on pots. The sleek stainless steel finish adds a modern touch to any kitchen, while the adjustable racks and spacious interior let you load more in fewer cycles. Plus, with smart connectivity, you can monitor and control your wash right from your phone — because clean dishes should never slow you down. "
]

# Create a Pandas DataFrame for your evaluation dataset
eval_dataset = pd.DataFrame({
    "system_prompt": system_prompt_data,
    "question": question_data,
    "description": description_data
})

print("--- Evaluation Dataset (first 3 rows) ---")
print(eval_dataset.head(3))
print("-" * 40)

# --- Define the Model to Evaluate ---
# Choose the model you want to evaluate. For example, "gemini-1.5-pro".
model = GenerativeModel("gemini-1.5-pro") # <--- You can change this to your desired model

# --- Create and Run the EvalTask ---
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY],
)

print(f"\n--- Starting Evaluation with model: {model._model_name} ---")
print(f"Using prompt template: '{prompt_template}'")

# Execute the evaluation
result = eval_task.evaluate(
    model=model,
    prompt_template=prompt_template,
    # Optional: Add an experiment_run_name for better tracking in Vertex AI Experiments
    experiment_run_name="product-summary-quality-eval"
)

# --- Display Results ---
print("\n--- Evaluation Metrics ---")
print(result.metrics)

print("\n--- Evaluation Results DataFrame (per-instance results) ---")
print(result.to_dataframe())